In [1]:
from foundations.projectables import ProjectableModel
from modeling.compute import create_vars
from numpy import pi
import numpy as np
from coupledgrad import pure_functions, coupled_functions
from coupledgrad import solver, concat, compose, eliminate, couplingvars
from coupledgrad import jacfwd, partial, jnp
np.set_printoptions(suppress=True)

In [2]:
A,V,S,W,Ww,CL,Re,Cf,CD,D = create_vars('A V S W W_w C_L Re C_f C_D D')
rho,mu,k,t,e,CDA,SwetS,W0 = 1.23, 1.78e-5, 1.2, 0.12, 0.96, 0.0306, 2.05, 4940

Aero = ProjectableModel()
CL = Aero.Var('C_L', 2*W/(rho*V**2*S))
Re = Aero.Var('Re', rho*V/mu*(S/A)**(1/2))
Cf = Aero.Var('C_f', 0.074/Re**0.2)
CD = Aero.Var('C_D', CDA/S+k*Cf*SwetS+CL**2/(pi*A*e))
D  = Aero.Var('D', 0.5*rho*V**2*CD*S)
Struct = ProjectableModel()
Ww = Struct.Var('Ww', 45.42*S+8.71e-5*2.5/t*A**(3/2)*S**(1/2)*(W0*W)**(1/2))
Struct.add_equation(W, W0+Ww)

In [3]:
eqs = [(p.right, p.var) for p in Aero.projectables+Struct.projectables]
Fdata = coupled_functions(eqs)
Rdata = pure_functions([right-left for right, left in eqs])
feed_forward, residual_functions, solvevars = couplingvars(Fdata, Rdata)

In [4]:
F,_ = compose(feed_forward)

In [5]:
F[2], F[1]

((S, V, W, A), (C_L, Re, C_f, C_D, D, Ww, S, V, W, A))

In [6]:
F[0]([10,30,6500,10])

DeviceArray([      1.17434508, 2073033.70786517,       0.00403562,
                   0.05871438,     324.9841024 ,    1482.4460987 ,
                  10.        ,      30.        ,    6500.        ,
                  10.        ], dtype=float64)

In [7]:
Rreduced = solver(residual_functions, solvevars, [[10000]], 
                eliminate=feed_forward)

In [8]:
Rreduced[0]([10,10,40])

array([19723.36507292])

In [9]:
F, R = compose((Rreduced,)+feed_forward, ((lambda D: D, (), (D,)),))
#objfunction = eliminate(Rdata, phi=Fdata)

In [10]:
F[0]([30,30,30])

DeviceArray([  24134.7887521 ,       1.45346515, 2073033.70786517,
                   0.00403562,       0.03429655,     569.49427885,
               19194.7887521 ,      30.        ,      30.        ,
                  30.        ], dtype=float64)

In [11]:
F[1],R[0]([10,40,10])

((W, C_L, Re, C_f, C_D, D, Ww, S, V, A),
 DeviceArray([[261.03749836]], dtype=float64))

In [12]:
from scipy.optimize import minimize
f = lambda x: R[0](x).item()
g = lambda x: np.array(jacfwd(R[0])(x)[0][0])
minimize(f, np.array([10,10,10]), 
            bounds=[(0,30),(0,50),(0,30)],
            jac=g)

      fun: 242.27476038088972
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00001557, -0.00000352,  0.00000003])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 19
      nit: 15
     njev: 19
   status: 0
  success: True
        x: array([ 5.25556812, 49.1827837 , 18.16319232])